In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn

# spacy for lemmatization
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import nltk

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\joblib\backports.py:22: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils  # noqa
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\multiclass.py:14: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\optimize.py:18: DeprecationWarning: Please use `line_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprec

In [3]:
df = pd.read_csv('D:\Projects\Jupyter\Github Docs\datasets\preprocessed_word_correct_token.csv', encoding='utf-8')
df.head()

<>:1: DeprecationWarning: invalid escape sequence '\P'
<>:1: DeprecationWarning: invalid escape sequence '\P'
C:\Users\researcher\AppData\Local\Temp\ipykernel_7976\1017671776.py:1: DeprecationWarning: invalid escape sequence '\P'
  df = pd.read_csv('D:\Projects\Jupyter\Github Docs\datasets\preprocessed_word_correct_token.csv', encoding='utf-8')


,id,index,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews,ngrams_token,lda_reviews,lda_token
0,275850,1,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...,"['review', 'prior', 'update', 'nothin', 'fail'...",review prior update nothin fail miserably anno...,"['review', 'prior', 'update', 'nothin', 'fail'..."
1,275850,2,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...,"['first', 'player', 'one', 'point', 'mear']",first come player one point mear,"['first', 'player', 'one', 'point', 'mear']"
2,275850,3,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...,"['prove', 'this_game', 'big', 'scam', 'know']",prove big scam know,"['prove', 'big', 'scam', 'know']"
3,275850,4,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...,"['the_developer', 'completely', 'lie_about', '...",developer completely lie countless feature meant,"['developer', 'completely', 'lie', 'countless'..."
4,275850,5,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...,"['clearly', 'strong', 'feel', 'play', 'hour', ...",clearly strong feel play hour hello games rele...,"['clearly', 'strong', 'feel', 'play_hour', 'he..."


In [4]:
df=df[:500]

In [5]:
eval(df['lda_token'][4])

['clearly',
 'strong',
 'feel',
 'play_hour',
 'hello_games',
 'release',
 'new',
 'update',
 'decide',
 'need',
 'resist',
 'order',
 'see',
 'sort',
 'improvement']

In [6]:
def convert(s):
    s = eval(s)
    s = ' '.join(s)
    return s
df['lda_token'] = df['lda_token'].apply(convert)

In [7]:
data = df.lda_token.values.tolist()
data[:3]

['review prior update nothin fail miserably annoyed stuck garbage library completely terrible likely never play',
 'first player one point mear',
 'prove big scam know']

# Vectorizer

## TF-IDF vectorizer

In [8]:
tfidf_vectorizer = TfidfVectorizer(
                                 max_df=500, 
                                   max_features=2000,
                                 min_df=0, 
#                                  stop_words='english',
                                   use_idf=True, 
#                                    token_pattern='[a-zA-Z0-9]{3,}', 
#                                    ngram_range=(1,3)
)

%time tfidf_vectorized = tfidf_vectorizer.fit_transform(data)

print(tfidf_vectorized.shape)

CPU times: total: 15.6 ms
Wall time: 99.4 ms
(500, 1405)


In [9]:
tfidf_data_vectorized = tfidf_vectorizer.fit_transform(data)

In [10]:
tfidf_vectorizer.get_feature_names()

['aaa',
 'abandon',
 'ability',
 'able',
 'absent',
 'absolutely',
 'abstract',
 'accept',
 'achieve',
 'achievement',
 'across',
 'action',
 'active',
 'activism',
 'actually',
 'add',
 'added',
 'addition',
 'additional',
 'address',
 'advance',
 'advanced',
 'advantage',
 'adversity',
 'advertised',
 'advertisement',
 'af',
 'afterward',
 'ago',
 'agree',
 'ahead',
 'ai',
 'aimless',
 'air',
 'algorithm',
 'aliasing',
 'alien',
 'alive',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'alpha',
 'already',
 'also',
 'alternate',
 'although',
 'always',
 'amain',
 'amaze',
 'ambitious',
 'amongst',
 'amount',
 'amp',
 'animal',
 'anisotropic',
 'annoy',
 'annoyed',
 'another',
 'anti',
 'anticipate',
 'anyone',
 'anything',
 'anything_like',
 'anyways',
 'anywhere',
 'apart',
 'apparent',
 'appeal',
 'appreciate',
 'approach',
 'appropriate',
 'argument',
 'armourer',
 'around',
 'arrive',
 'art',
 'artstyle',
 'artsy',
 'aside',
 'ask',
 'aspect',
 'assumption',
 'atlas',
 'atom

In [11]:
tfidf_data_feature_names=tfidf_vectorizer.get_feature_names()

In [12]:
tfidf_data_dense = tfidf_data_vectorized.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("tfidf_Sparsicity: ", ((tfidf_data_dense > 0).sum()/tfidf_data_dense.size)*100, "%")
# Since most cells in this matrix will be zero, I am interested in knowing what percentage of cells contain non-zero values.

tfidf_Sparsicity:  0.586761565836299 %


# Build LDA Model

In [13]:
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(tfidf_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [14]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)

In [15]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(tfidf_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(tfidf_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -14809.463044918686
Perplexity:  78934.75912888303
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [16]:
# Define Search Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 'max_iter': [5, 10, 15, 20, 30, 40, 50]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(tfidf_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'max_iter': [5, 10, 15, 20, 30, 40, 50],
                         'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45,
                                          50]})

In [17]:
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
        n_jobs=1,
       param_grid={'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 'max_iter': [5, 10, 15, 20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'max_iter': [5, 10, 15, 20, 30, 40, 50],
                         'n_components': [5, 10, 15, 20, 25, 30, 35, 40, 45,
                                          50]},
             return_train_score='warn')

In [18]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(tfidf_vectorized))

Best Model's Params:  {'max_iter': 50, 'n_components': 5}
Best Log Likelihood Score:  -3577.0264119927756
Model Perplexity:  7553.944872666494
